In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline, set_seed
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import tensorflow as tf
from transformers import TFGPT2LMHeadModel
from sklearn.model_selection import train_test_split

2024-04-20 16:05:57.082129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 16:05:57.082177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 16:05:57.083570: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-20 16:05:57.090513: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-20 16:05:57.903874: W tensorflow/compiler/tf2

In [2]:
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [7]:
Reviews = pd.read_csv('/home/orchid/Documents/HumourDetection/Sarthak_Sharma/IR_ASSIGNMENT_4/Reviews.csv')

In [8]:
Reviews

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [9]:
Reviews.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

PRE PROCESSING THE SUMMARY AND TEXT COLUMN

In [10]:
#Processing the data and dropping irrelevant columns for our study

Reviews.drop(columns=['Time','Score','ProfileName','UserId','ProductId','HelpfulnessDenominator','HelpfulnessNumerator'], inplace=True)

In [11]:
#We will work with this data from now

Reviews

,Id,Summary,Text
0,1,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,Cough Medicine,If you are looking for the secret ingredient i...
4,5,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...
568449,568450,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [12]:
#Total na or null values in both the 'Summary' and 'Text' columns

Reviews[['Summary','Text']].isna().sum()

Summary    27
Text        0
dtype: int64

In [13]:
#Filling the na values with empty string

Reviews['Summary'] = Reviews['Summary'].fillna('')

In [14]:
# Finally we have the concerned data without null values
Reviews[['Id','Summary','Text']].isna().sum()

Id         0
Summary    0
Text       0
dtype: int64

In [15]:
#Checking for duplicates 
Reviews[['Summary','Text']].duplicated().sum()

173484

Applying all the Pre-Processing steps for both the 'Summary' and 'Text' columns :

1. Removing Duplicate values
2. Removing HTML Tags
3. Tokenize
4. Punctuation Removed
5. Stopwords Removed
6. Expanding acronyms (if any)
7. Stemming
8. Lemmatization
9. Normalizing the text data by using lowercase

In [17]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from tqdm import tqdm
from bs4 import BeautifulSoup

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Remove duplicates
Reviews.drop_duplicates(subset=['Text', 'Summary'], inplace=True)

# Define function for HTML tag removal
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

acronyms = {
    'LOL': 'Laughing Out Loud - Used to indicate laughter or amusement.',
    'BRB': 'Be Right Back - Used to indicate that the person will return shortly.',
    'OMG': 'Oh My God - Used to express surprise, shock, or amazement.',
    'ICYMI': 'In Case You Missed It - Used to bring attention to something that someone may have missed.',
    'AMA': 'Ask Me Anything - Used to invite others to ask questions about anything.',
    'DM': 'Direct Message - A private message sent on social media platforms or messaging apps.',
    'ETA': 'Estimated Time of Arrival - Used to indicate when someone or something is expected to arrive.',
    'ETA': 'Edited To Add - Used to indicate that a post has been edited to include additional information.',
    'FTW': 'For The Win - Used to express enthusiasm or support for something.',
    'FWIW': 'For What It\'s Worth - Used to preface a statement that may not be very valuable or important.',
    'JK': 'Just Kidding - Used to indicate that a previous statement was made in jest or humor.',
    'NP': 'No Problem - Used to indicate that something was easily done or accomplished.',
    'TY': 'Thank You - Used to express gratitude.',
    'YW': 'You\'re Welcome - Used as a response to "thank you" to acknowledge the thanks.',
    'GG': 'Good Game - Used to acknowledge the end of a game or competition, often with respect for the opponents.',
    'GG': 'Gotta Go - Used to indicate that the person needs to leave or end the conversation.',
     'IMO': 'In My Opinion - Used to preface a statement expressing one\'s personal viewpoint.',
    'IMHO': 'In My Humble Opinion - Similar to IMO, used to express a personal opinion with humility.',
    'TBH': 'To Be Honest - Used to preface a statement that may be candid or blunt.',
    'FYI': 'For Your Information - Used to provide information that may be useful or relevant.',
    'BTW': 'By The Way - Used to introduce additional or incidental information.',
    'ICYMI': 'In Case You Missed It - Used to bring attention to something that someone may have missed.',
    'TMI': 'Too Much Information - Used to indicate that someone has shared more information than necessary or appropriate.',
    'IDK': 'I Don\'t Know - Used to indicate uncertainty or lack of knowledge.',
    'AFK': 'Away From Keyboard - Used to indicate that the person is temporarily unavailable.',
    'IRL': 'In Real Life - Used to distinguish between online interactions and interactions in the physical world.',
    'AMA': 'Ask Me Anything - Used to invite others to ask questions about anything.',
    'DM': 'Direct Message - A private message sent on social media platforms or messaging apps.',
    'ETA': 'Estimated Time of Arrival - Used to indicate when someone or something is expected to arrive.',
    'ETA': 'Edited To Add - Used to indicate that a post has been edited to include additional information.',
    'FTW': 'For The Win - Used to express enthusiasm or support for something.',
}

# Define function for acronym expansion
def expand_acronyms(text):
    for acronym, expansion in acronyms.items():
        text = re.sub(r'\b' + re.escape(acronym) + r'\b', expansion, text)
    return text

# Define function for text preprocessing
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Remove HTML tags
    text = remove_html_tags(text)
    # Tokenization
    words = word_tokenize(text)
    # Remove punctuations
    words = [word for word in words if word.isalpha()]
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    # Expand acronyms
    text = expand_acronyms(' '.join(words))
    # Stemming
    words = [stemmer.stem(word) for word in words]
    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    # Normalize tokens to lowercase
    words = [word.lower() for word in words]
    # Join words back into sentence
    return ' '.join(words)

# Apply text preprocessing to 'Text' and 'Summary' columns
tqdm.pandas()
Reviews['Text'] = Reviews['Text'].progress_apply(preprocess_text)
Reviews['Summary'] = Reviews['Summary'].progress_apply(preprocess_text)


[nltk_data] Downloading package punkt to /home/orchid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/orchid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/orchid/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
 16%|█████▍                            | 63095/394970 [00:11<01:00, 5504.86it/s]/usr/lib/python3/dist-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://www.amazon.com/gp/product/B007I7YYGY/ref=cm_cr_rev_prod_title" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
 31%|██████████▎                      | 123006/394970 [00:22<00:49, 5509.62it/s]/usr/lib/python3/dist-packages/bs4/__init__.py:337: MarkupResemblesLocatorWarning: "." looks like a directory name, 

In [19]:
#Clean and Pre-Processed data

Reviews.head(50)

,Id,Summary,Text
0,1,good qualiti dog food,i bought sever vital can dog food product foun...
1,2,not advertis,product arriv label jumbo salt peanut peanut a...
2,3,delight say,thi confect around centuri it light pillowi ci...
3,4,cough medicin,if look secret ingredi robitussin i believ i f...
4,5,great taffi,great taffi great price there wide assort yumm...
5,6,nice taffi,i got wild hair taffi order five pound bag the...
6,7,great just good expens brand,thi saltwat taffi great flavor soft chewi each...
7,8,wonder tasti taffi,thi taffi good it soft chewi the flavor amaz i...
8,9,yay barley,right i mostli sprout cat eat grass they love ...
9,10,healthi dog food,thi healthi dog food good digest also good sma...


In [21]:
Reviews.isnull().sum()

Id         0
Summary    0
Text       0
dtype: int64

In [22]:
Reviews = Reviews[['Text','Summary']]

In [23]:
Reviews

,Text,Summary
0,i bought sever vital can dog food product foun...,good qualiti dog food
1,product arriv label jumbo salt peanut peanut a...,not advertis
2,thi confect around centuri it light pillowi ci...,delight say
3,if look secret ingredi robitussin i believ i f...,cough medicin
4,great taffi great price there wide assort yumm...,great taffi
...,...,...
568449,great sesam chicken good better restur i eaten...,will without
568450,i disappoint flavor the chocol note especi wea...,disappoint
568451,these star small give one train session i tri ...,perfect maltipoo
568452,these best treat train reward dog good groom l...,favorit train reward treat


In [24]:
# Save cleaned and preprocessed dataset
Reviews.to_csv('/home/orchid/Documents/HumourDetection/Sarthak_Sharma/IR_ASSIGNMENT_4/processed_reviews.csv', index=False)

In [3]:
Reviews = pd.read_csv('/home/orchid/Documents/HumourDetection/Sarthak_Sharma/IR_ASSIGNMENT_4/processed_reviews.csv')

In [4]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer

Processed_Reviews = Reviews.head(15000)

# Divide the Subset into Training and Testing Sets
train_df, test_df = train_test_split(Processed_Reviews, test_size=0.25, random_state=42)

# Define the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Create a Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx]['Text'])  # Convert to string
        summary = str(self.data.iloc[idx]['Summary'])  # Convert to string

        encoded_text = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=80)
        encoded_summary = tokenizer(summary, return_tensors='pt', padding='max_length', truncation=True, max_length=80)

        return {
            'input_ids': encoded_text['input_ids'].squeeze(),
            'attention_mask': encoded_text['attention_mask'].squeeze(),
            'labels': encoded_summary['input_ids'].squeeze(),
            'Summary': summary  # Include the 'Summary' column in the returned dictionary
        }

# Create instances of the Custom Dataset Class for Training and Testing
train_dataset = CustomDataset(train_df)
test_dataset = CustomDataset(test_df)

# Create DataLoaders for Training and Testing
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Example usage of the DataLoader
for batch in train_dataloader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    # Perform training steps using input_ids, attention_mask, and labels

# Similarly, you can iterate over test_dataloader for testing.


In [5]:
import torch
from transformers import GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup

# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

torch.cuda.set_device(0)
# Set the device for the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the number of epochs
NUM_EPOCHS = 3

# Define the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define the number of training steps
num_training_steps = len(train_dataloader) * NUM_EPOCHS

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

# Define the training function
def train(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0.0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

# Training loop
for epoch in range(NUM_EPOCHS):
    train_loss = train(model, train_dataloader, optimizer, scheduler, device)
    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Train Loss: {train_loss}")

# Save the fine-tuned model
model.save_pretrained("/home/orchid/Documents/HumourDetection/Sarthak_Sharma/IR_ASSIGNMENT_4/fine_tuned_gpt2_model")


/home/orchid/.local/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Train Loss: 0.7614301108670506
Epoch 2/3, Train Loss: 0.34872168476100673
Epoch 3/3, Train Loss: 0.3421756595542485


In [8]:
# Define the training function
'''def train(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0.0
    for batch in tqdm(dataloader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss
'''
# Define the hyperparameters to experiment with
learning_rates = [5e-5, 3e-5, 1e-5]  # Example learning rates
batch_sizes = [16, 32]  # Example batch sizes
num_epochs_list = [3, 5]  # Example number of epochs

# Training loop with different hyperparameters
for lr in learning_rates:
    for bs in batch_sizes:
        for epochs in num_epochs_list:
            # Load the pre-trained GPT-2 model
            model = GPT2LMHeadModel.from_pretrained("gpt2")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)

            # Define the optimizer and scheduler
            optimizer = AdamW(model.parameters(), lr=lr)
            scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=len(train_dataloader) * epochs)

            # Training loop
            for epoch in range(epochs):
                train_loss = train(model, train_dataloader, optimizer, scheduler, device)
                print(f"Epoch {epoch + 1}/{epochs}, Learning Rate: {lr}, Batch Size: {bs}, Train Loss: {train_loss}")

            # Save the fine-tuned model
            model.save_pretrained(f"/home/orchid/Documents/HumourDetection/Sarthak_Sharma/IR_ASSIGNMENT_4/fine_tuned_gpt2_model_lr_{lr}_bs_{bs}_epochs_{epochs}")


Epoch 1/3, Learning Rate: 5e-05, Batch Size: 16, Train Loss: 0.7631358192725615
Epoch 2/3, Learning Rate: 5e-05, Batch Size: 16, Train Loss: 0.34908453074537893
Epoch 3/3, Learning Rate: 5e-05, Batch Size: 16, Train Loss: 0.3423381774601611


/home/orchid/.local/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Learning Rate: 5e-05, Batch Size: 16, Train Loss: 0.7556755510582165
Epoch 2/5, Learning Rate: 5e-05, Batch Size: 16, Train Loss: 0.34771051770076156
Epoch 3/5, Learning Rate: 5e-05, Batch Size: 16, Train Loss: 0.340826706241139
Epoch 4/5, Learning Rate: 5e-05, Batch Size: 16, Train Loss: 0.33764659583737905
Epoch 5/5, Learning Rate: 5e-05, Batch Size: 16, Train Loss: 0.335888842840425


/home/orchid/.local/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Learning Rate: 5e-05, Batch Size: 32, Train Loss: 0.7593160339377143
Epoch 2/3, Learning Rate: 5e-05, Batch Size: 32, Train Loss: 0.3486774725500833
Epoch 3/3, Learning Rate: 5e-05, Batch Size: 32, Train Loss: 0.34222007322717796


...

Epoch 1/5, Learning Rate: 1e-05, Batch Size: 16, Train Loss: 1.1953012144870379
Epoch 2/5, Learning Rate: 1e-05, Batch Size: 16, Train Loss: 0.39868673284284095
Epoch 3/5, Learning Rate: 1e-05, Batch Size: 16, Train Loss: 0.37440803105180914
Epoch 4/5, Learning Rate: 1e-05, Batch Size: 16, Train Loss: 0.36363982722501864
Epoch 5/5, Learning Rate: 1e-05, Batch Size: 16, Train Loss: 0.3592761482789435


/home/orchid/.local/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Learning Rate: 1e-05, Batch Size: 32, Train Loss: 1.200513393448835
Epoch 2/3, Learning Rate: 1e-05, Batch Size: 32, Train Loss: 0.40409081505442207
Epoch 3/3, Learning Rate: 1e-05, Batch Size: 32, Train Loss: 0.38578693179244344


/home/orchid/.local/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Learning Rate: 1e-05, Batch Size: 32, Train Loss: 1.195465607229959
Epoch 2/5, Learning Rate: 1e-05, Batch Size: 32, Train Loss: 0.3988317939537493
Epoch 3/5, Learning Rate: 1e-05, Batch Size: 32, Train Loss: 0.37440009796145285
Epoch 4/5, Learning Rate: 1e-05, Batch Size: 32, Train Loss: 0.3635590917176821
Epoch 5/5, Learning Rate: 1e-05, Batch Size: 32, Train Loss: 0.3592039352198216


In [9]:
# Iterate over a single batch from the DataLoader
for batch in test_dataloader:
    # Print the keys of the batch
    print(batch.keys())
    # Break after printing keys of the first batch
    break


dict_keys(['input_ids', 'attention_mask', 'labels', 'Summary'])


In [10]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import DataLoader
from rouge import Rouge
from tqdm import tqdm

# Load the saved fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/home/orchid/Documents/HumourDetection/Sarthak_Sharma/IR_ASSIGNMENT_4/fine_tuned_gpt2_model/")
#tokenizer = GPT2Tokenizer.from_pretrained("/home/orchid/Documents/HumourDetection/Sarthak_Sharma/IR_ASSIGNMENT_4/fine_tuned_gpt2_model/")

torch.cuda.set_device(0)
# Set the device for the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a function to generate summaries from the model
def generate_summaries(model, tokenizer, dataloader, device):
    summaries = []
    references = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Generating Summaries"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Generate summaries
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=100, num_beams=4, early_stopping=True)

            # Decode generated summaries
            decoded_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            summaries.extend(decoded_summaries)

            # Get the references (actual summaries)
            references.extend([str(summary) for summary in batch['Summary']])

    return summaries, references

# Load the test dataset and create a DataLoader
test_dataset = CustomDataset(test_df)
print("Test Data",test_dataset)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Generate summaries for the test set
predicted_summaries, actual_summaries = generate_summaries(model, tokenizer, test_dataloader, device)

# Compute ROUGE scores
rouge = Rouge()
scores = rouge.get_scores(predicted_summaries, actual_summaries, avg=True)

# Print the ROUGE scores
print("ROUGE Scores:")
print(scores)


Test Data <__main__.CustomDataset object at 0x7c8b57508b50>


Generating Summaries:   0%|                             | 0/118 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Generating Summaries:   1%|▏                    | 1/118 [00:00<00:42,  2.76it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Generating Summaries:   2%|▎                    | 2/118 [00:00<00:38,  3.05it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Generating Summaries:   3

ROUGE Scores:
{'rouge-1': {'r': 0.43619882487882433, 'p': 0.056787406295847054, 'f': 0.09652970097920145}, 'rouge-2': {'r': 0.12342678506678481, 'p': 0.012133117663315379, 'f': 0.021025393913372086}, 'rouge-l': {'r': 0.40230660450660466, 'p': 0.052011204349104036, 'f': 0.08846398662736299}}


In [12]:
print("ROUGE Scores:")
for metric, values in scores.items():
    precision = values['p']
    recall = values['r']
    f1_score = values['f']
    print(f"{metric.upper()}: Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1_score:.2f}")


ROUGE Scores:
ROUGE-1: Precision: 0.06, Recall: 0.44, F1-Score: 0.10
ROUGE-2: Precision: 0.01, Recall: 0.12, F1-Score: 0.02
ROUGE-L: Precision: 0.05, Recall: 0.40, F1-Score: 0.09


In [26]:
# Input text
input_text = "I have been drinking this tea for a long time now.  \
I used to have to purchase it at a doctor's office because it wasn't available elsewhere. \
I'm so glad that I can buy it now from Amazon.com.  I drink this tea throughout the day like other folks drink coffee.  Wonderful taste."

# Tokenize and preprocess the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate summary
generated_summary = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)

# Decode the generated summary
decoded_summary = tokenizer.decode(generated_summary[0], skip_special_tokens=True)

print("Generated Summary:", decoded_summary)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Summary: I have been drinking this tea for a long time now.  I used to have to purchase it at a doctor's office because it wasn't available elsewhere. I'm so glad that I can buy it now from Amazon.com.  I drink this tea throughout the day like other folks drink coffee.  Wonderful taste.


In [19]:
proc_review = pd.read_csv('/home/orchid/Documents/HumourDetection/Sarthak_Sharma/IR_ASSIGNMENT_4/Reviews.csv')

In [21]:
proc_review['Text'][111]

"I have been drinking this tea for a long time now.  I used to have to purchase it at a doctor's office because it wasn't available elsewhere.  I'm so glad that I can buy it now from Amazon.com.  I drink this tea throughout the day like other folks drink coffee.  Wonderful taste."